<a href="https://colab.research.google.com/github/proxtor/-Salon-Appointment-Scheduler/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Part I: Convert categorical data to numbers
dataset['sex'] = dataset['sex'].astype('category').cat.codes
dataset['smoker'] = dataset['smoker'].astype('category').cat.codes
dataset['region'] = dataset['region'].astype('category').cat.codes

In [ ]:
# Part II: Split data into train & test dataset

# Get randomly 80% of data for the train dataset
train_dataset = dataset.sample(frac=0.8)
# Get the rest of the data for the test dataset
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
# Part III: Separate train & test labels from train & test datasets

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# Part IV: Build model

# Create a normalization layer adapted to the dataset
from tensorflow.keras.layers import Normalization
normalizer = Normalization()
normalizer.adapt(np.array(train_dataset))

# Build model using normalization layer and dense layers
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=6, activation='relu'),
    layers.Dense(units=6, activation='relu'),
    layers.Dense(units=1)
])
# Show summary
model.summary()

In [ ]:
# Part V: Compile and fit model

# Compile model with proper metrics to evaluate for later
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error',
    metrics=['mae', 'mse'])

# Fit model and get training history
history = model.fit(
    # Dataset and labels to use for training
    train_dataset, train_labels,
    # Number of epochs
    epochs=100,
    # Suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

# Show history in the last few epochs
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
